In [1]:
import os

PATH = os.path.dirname(os.path.abspath(__name__))
NEW_PATH = str.replace(PATH, "notebooks", "")
os.chdir(NEW_PATH)

# Obtendo Dados

In [19]:

from dags.apps.crawlers import Crawler

In [20]:
crawler: Crawler = Crawler()
crawler.main()

# EDA

In [2]:
# importando os dados
import pandas as pd

df_ibge = pd.read_csv("data/raw/ibge.csv", sep=",")
df_pnad = pd.read_csv("data/raw/pnad.csv", sep=",")

In [3]:
# filtrando os dados da PNAD
df_pnad_filtered = df_pnad.loc[
    (df_pnad.sexo == 'Mulher') & 
    (df_pnad.idade >= 20) & 
    (df_pnad.idade <= 40)
]

# ETL

In [93]:
import unidecode 
# !pip install unidecode

In [94]:
# limpando as colunas de e padronizando os textos
df1 = df_pnad_filtered.copy()
df2 = df_ibge.copy()

f = lambda x: unidecode.unidecode(str(x).strip().upper())
df1['uf'] = df1['uf'].apply(f)
df1['cor'] = df1['cor'].apply(f)
df1['sexo'] = df1['sexo'].apply(f)
df1['graduacao'] = df1['graduacao'].apply(f)
df1['trab'] = df1['trab'].apply(f)
df1['ocup'] = df1['ocup'].apply(f)
df2['nome'] = df2['nome'].apply(f)

In [95]:
# removendo colunas desnecessárias
df3 = df1.drop(columns=['_id', 'anosesco'])


In [96]:
# Criando a tabela estados
estados = df2[['id','sigla','nome','regiao_id']]
estados = estados.drop_duplicates()

In [97]:
# Criando a tabela regioes
regioes = df2[['regiao_id','regiao_nome']]
regioes = regioes.drop_duplicates()

In [98]:
# buscando o id do estado
df4 = df3.merge(estados, left_on='uf', right_on='nome', how='left')
# df4.head()


In [ ]:
df = pd.read_csv("data/raw/ibge.csv", sep=",")

In [99]:
# criando a tabela pnads
pnads = df4[['ano', 'trimestre', 'idade','cor','sexo','graduacao','trab','ocup','renda','horastrab','id']]

# renomeando as colunas
pnads = pnads.rename(columns={
    'id':'estado_id',
    'trab':'tipo_trabalhador',
    'ocup':'ocupacao'
    })

In [15]:
df_pnad = pd.read_csv("data/raw/pnad.csv", sep=",")
dim_cores = pd.read_csv("data/processed/dim_cores.csv", sep=",")
dim_estados = pd.read_csv("data/processed/dim_estados.csv", sep=",")
dim_regioes = pd.read_csv("data/processed/dim_regioes.csv", sep=",")
dim_tempo = pd.read_csv("data/processed/dim_tempo.csv", sep=",")

In [8]:
df_pnad.head(5)

,_id,ano,trimestre,uf,sexo,idade,cor,graduacao,trab,ocup,renda,horastrab,anosesco,data_criacao
0,600ce0685daa95c6616007b1,2020,3,ACRE,MULHER,35,PARDA,NAN,PESSOAS NA FORCA DE TRABALHO,PESSOAS OCUPADAS,800.0,36.0,16.0,2022-10-23
1,600ce0685daa95c6616007db,2020,3,ACRE,MULHER,40,PARDA,NAN,PESSOAS FORA DA FORCA DE TRABALHO,NAN,NaN,NaN,12.0,2022-10-23
2,600ce0685daa95c6616007bf,2020,3,ACRE,MULHER,30,BRANCA,NAN,PESSOAS NA FORCA DE TRABALHO,PESSOAS OCUPADAS,1049.0,44.0,16.0,2022-10-23
3,600ce0685daa95c6616007cd,2020,3,ACRE,MULHER,38,PARDA,NAN,PESSOAS FORA DA FORCA DE TRABALHO,NAN,NaN,NaN,16.0,2022-10-23
4,600ce0685daa95c6616007d9,2020,3,ACRE,MULHER,31,BRANCA,NAN,PESSOAS NA FORCA DE TRABALHO,PESSOAS OCUPADAS,1200.0,44.0,14.0,2022-10-23


In [9]:
dim_cores.head()

,cor,data_criacao,id_cor
0,PARDA,2022-10-23,2
1,BRANCA,2022-10-23,9
2,PRETA,2022-10-23,223
3,INDIGENA,2022-10-23,605
4,AMARELA,2022-10-23,6037


In [12]:
dim_estados.head()

,id,sigla,nome,regiao_id,data_criacao
0,11,RO,RONDONIA,1,2022-10-23
1,12,AC,ACRE,1,2022-10-23
2,13,AM,AMAZONAS,1,2022-10-23
3,14,RR,RORAIMA,1,2022-10-23
4,15,PA,PARA,1,2022-10-23


In [54]:
# buscando a cor
df = df_pnad.merge(dim_cores, left_on='cor', right_on='cor', how='left')

In [55]:
# df = df.merge(dim_estados, left_on='uf', right_on='nome', how='left')

In [56]:
# busca estados
df = df.merge(dim_estados, left_on='uf', right_on='nome', how='left')

In [57]:
dim_tempo.head(5)

,data,ano,trimestre,id_calendario
0,2000-01-01,2000,1,2001
1,2000-04-01,2000,2,2002
2,2000-07-01,2000,3,2003
3,2000-10-01,2000,4,2004
4,2001-10-01,2001,4,2005


In [58]:
# cria a sk_data
f = lambda x: int(str(x['trimestre'])+str(x['ano']))
df['sk_tempo'] = df.apply(f, axis=1)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57335 entries, 0 to 57334
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _id             57335 non-null  object 
 1   ano             57335 non-null  int64  
 2   trimestre       57335 non-null  int64  
 3   uf              57335 non-null  object 
 4   sexo            57335 non-null  object 
 5   idade           57335 non-null  int64  
 6   cor             57335 non-null  object 
 7   graduacao       57335 non-null  object 
 8   trab            57335 non-null  object 
 9   ocup            57335 non-null  object 
 10  renda           26792 non-null  float64
 11  horastrab       28012 non-null  float64
 12  anosesco        57335 non-null  float64
 13  data_criacao_x  57335 non-null  object 
 14  data_criacao_y  57335 non-null  object 
 15  id_cor          57335 non-null  int64  
 16  id              57335 non-null  int64  
 17  sigla           57335 non-null 

In [60]:
# filtrando as colunas
mask = ['sk_data', 'id_cor', 'id']
df1 = df[mask]

In [61]:
df['graduacao'].value_counts()

NAN    51340
NAO     5441
SIM      554
Name: graduacao, dtype: int64

In [62]:
df['graduacao'].replace(
    {'SIM':1,
    'NAO': pd.NA,
    'NAN': pd.NA}, inplace=True)

In [67]:
df['ocup'].replace(
    {'PESSOAS OCUPADAS':1,
    'PESSOAS DESOCUPADAS': pd.NA,
    'NAN': pd.NA}, inplace=True)

In [71]:
df['renda'].mean()

1809.284263959391

In [70]:
df['horastrab'].mean()

33.66560759674425

In [76]:
mask = ['sk_data', 'id_cor', 'id']
fato_pnad = df.groupby(mask, as_index=False).agg(
    {
    '_id':'count',
    'graduacao': 'sum',
    'ocup': 'sum',
    'renda': 'mean',
    'horastrab': 'mean'})

In [78]:
fato_pnad.fillna(0, inplace=True)

In [80]:
fato_pnad.rename(columns={
    'id_cor': 'sk_cor',
    'id': 'sk_estado',
    '_id': 'qtd_pessoas',
    'graduacao': 'qtd_graduacao',
    'ocup': 'qtd_ocupados',
    'renda': 'renda_media',
    'horastrab': 'horastrab_media'}, inplace=True)

In [81]:
fato_pnad

,sk_data,sk_cor,sk_estado,qtd_pessoas,qtd_graduacao,qtd_ocupados,renda_media,horastrab_media
0,32020,2,11,645,6,310,1543.951220,34.951613
1,32020,2,12,851,8,314,1605.186275,31.394904
2,32020,2,13,1482,8,676,1429.670796,32.170118
3,32020,2,14,652,8,274,1626.836431,33.832117
4,32020,2,15,1605,13,697,1289.337559,31.116212
...,...,...,...,...,...,...,...,...
132,32020,49481,31,1,0,1,1045.000000,30.000000
133,32020,49481,33,1,0,1,2000.000000,30.000000
134,32020,49481,42,1,0,0,0.000000,0.000000
135,32020,49481,50,2,0,1,400.000000,98.000000
